In [1]:
import spox.opset.ai.onnx.v17 as op
from spox import argument, build, inline, Tensor
import tensorflow as tf
import os
import tf2onnx
import onnx
import onnxruntime as rt
import numpy as np
# from keras.layers import Lambda

2025-01-29 11:49:21.969322: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load the keras model

In [2]:
main_dir='/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/keras_models_morphing'

#read all h5 files in the directory
h5_files = [x for x in os.listdir(main_dir) if x.endswith('.h5')]
print(h5_files)



model = tf.keras.models.load_model(main_dir+'/'+h5_files[0])
print(model.summary())
print(model.name)
print(model.input_shape)


# for layer in model.layers:
#     print(layer.get_output_at(0).get_shape().as_list())


['model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold1.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold2.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold3.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold4.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold5.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold6.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold7.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold8.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold9.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold10.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold11.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold12.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold13.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold14.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold15.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold16.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold17.h5', 'model_reweightingTo4b_PASv2p0_sigmaMbb

In [3]:
# lambda_func=Lambda(lambda inputs: inputs[0] / inputs[1])([model.layers[-1].output[1],model.layers[-1].output[0]])

In [4]:
model.layers[0].output

<KerasTensor: shape=(None, 45) dtype=float32 (created by layer 'batch_normalization')>

# get the ratio of the probability of the two classes

In [5]:
#modify the model to get as output the ratio of the two classes probabilities
# model_ratio = tf.keras.models.Model(inputs=model.input, outputs=model.layers[-1].output[1]/model.layers[-1].output[0])
# model_ratio = tf.keras.models.Model(inputs=model.input, outputs= tf.math.divide(model.layers[-1].output[1],model.layers[-1].output[0]))
# model_ratio = tf.keras.models.Model(inputs=model.layers[0].output, outputs= lambda_func)
model_ratio = tf.keras.models.Model(inputs=model.input, outputs= model.output[:,1]/model.output[:,0])
print(model_ratio.summary())
print(model_ratio.input_shape)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 batch_normalization_input   [(None, 45)]                 0         []                            
 (InputLayer)                                                                                     
                                                                                                  
 batch_normalization (Batch  (None, 45)                   180       ['batch_normalization_input[0]
 Normalization)                                                     [0]']                         
                                                                                                  
 dense (Dense)               (None, 512)                  23552     ['batch_normalization[0][0]'] 
                                                                                              

                                                                                                  
 dense_1 (Dense)             (None, 256)                  131328    ['dropout[0][0]']             
                                                                                                  
 batch_normalization_2 (Bat  (None, 256)                  1024      ['dense_1[0][0]']             
 chNormalization)                                                                                 
                                                                                                  
 dropout_1 (Dropout)         (None, 256)                  0         ['batch_normalization_2[0][0]'
                                                                    ]                             
                                                                                                  
 dense_2 (Dense)             (None, 128)                  32896     ['dropout_1[0][0]']           
          

# name of the input columns

In [6]:

columns =   ["era","higgs1_reco_pt","higgs1_reco_eta","higgs1_reco_phi","higgs1_reco_mass",
            "higgs2_reco_pt","higgs2_reco_eta","higgs2_reco_phi","higgs2_reco_mass",
            "HT","higgs1_DeltaRjj","higgs2_DeltaRjj","minDeltaR_Higgjj","maxDeltaR_Higgjj",
            "higgs1_helicityCosTheta","higgs2_helicityCosTheta","hh_CosThetaStar_CS",
            "hh_vec_mass","hh_vec_pt","hh_vec_eta","hh_vec_DeltaR","hh_vec_DeltaPhi","hh_vec_DeltaEta",
            "higgs1_reco_jet1_pt","higgs1_reco_jet1_eta","higgs1_reco_jet1_phi","higgs1_reco_jet1_mass",
            "higgs1_reco_jet2_pt","higgs1_reco_jet2_eta","higgs1_reco_jet2_phi","higgs1_reco_jet2_mass",
            "higgs2_reco_jet1_pt","higgs2_reco_jet1_eta","higgs2_reco_jet1_phi","higgs2_reco_jet1_mass",
            "higgs2_reco_jet2_pt","higgs2_reco_jet2_eta","higgs2_reco_jet2_phi","higgs2_reco_jet2_mass",
            "add_jet1pt_pt","add_jet1pt_eta","add_jet1pt_phi","add_jet1pt_mass",
            "sigma_over_higgs1_reco_mass","sigma_over_higgs2_reco_mass"]
print(len(columns))

45


# Test on fake event

In [7]:
# create a dummy input
input_data = [1]+[0.01]*44

input_tensor = tf.convert_to_tensor(input_data, dtype=tf.float32)
# add a none dimension to the input tensor
input_tensor = tf.expand_dims(input_tensor, 0)
print(input_tensor.shape)
pred = model.predict(input_tensor)
print(pred)

(1, 45)
1/1 [==============================] - 0s 239ms/step
[[0.1170899 0.8829101]]


In [8]:
pred_ratio = model_ratio.predict(input_tensor)
print(pred_ratio)

1/1 [==============================] - 0s 164ms/step
[7.540446]


In [9]:
0.8829101/0.1170899

7.540446272479523

# Convert to onnx

In [10]:
onnx_model,_=tf2onnx.convert.from_keras(model_ratio, input_signature=[tf.TensorSpec(shape=(None, 45), dtype=tf.float32)])
# print(type(onnx_model), onnx_model)
onnx.save(onnx_model, "./model.onnx")

2025-01-29 11:49:27.086170: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2025-01-29 11:49:27.086333: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-01-29 11:49:27.237769: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2025-01-29 11:49:27.237915: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


## load the onnx model and test if the result is the same as for the keras model

In [11]:
session = rt.InferenceSession(
    "./model.onnx",
    providers=rt.get_available_providers()
)

# print the input/putput name and shape
input_name=[input.name for input in session.get_inputs()]
output_name=[output.name for output in session.get_outputs()]
print("Inputs name:", input_name)
print("Outputs name:", output_name)

input_shape=[input.shape for input in session.get_inputs()]
output_shape=[output.shape for output in session.get_outputs()]
print("Inputs shape:", input_shape)
print("Outputs shape:", output_shape)


Inputs name: ['args_0']
Outputs name: ['tf.math.truediv']
Inputs shape: [['unk__18', 45]]
Outputs shape: [['unk__19']]


In [12]:
input_example = {input_name[0]: [input_data]}
print(input_example)

output = session.run(output_name, input_example)
print(output)

{'args_0': [[1, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]]}
[array([7.5404353], dtype=float32)]


# load a different model

In [13]:
model1 = tf.keras.models.load_model(main_dir+'/'+h5_files[1])
pred1 = model1.predict(input_tensor)
print(pred1)
model_ratio1 = tf.keras.models.Model(inputs=model1.input, outputs= model1.output[:,1]/model1.output[:,0])
pred_ratio1 = model_ratio1.predict(input_tensor)
print(pred_ratio1)


1/1 [==============================] - 0s 132ms/step
[[9.994678e-01 5.321892e-04]]
1/1 [==============================] - 0s 139ms/step
[0.00053247]


In [14]:
5.321892e-04/9.994678e-01


0.0005324725819080915

In [15]:

onnx_model1,_=tf2onnx.convert.from_keras(model_ratio1, input_signature=[tf.TensorSpec(shape=(None, 45), dtype=tf.float32)])


2025-01-29 11:49:28.282802: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2025-01-29 11:49:28.282909: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-01-29 11:49:28.417776: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2025-01-29 11:49:28.417881: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


In [23]:
model2 = tf.keras.models.load_model(main_dir+'/'+h5_files[2])
pred2 = model2.predict(input_tensor)
print(pred2)
model_ratio2 = tf.keras.models.Model(inputs=model2.input, outputs= model2.output[:,1]/model2.output[:,0])
pred_ratio2 = model_ratio2.predict(input_tensor)
print(pred_ratio2)


1/1 [==============================] - 0s 135ms/step
[[0.811733   0.18826701]]
1/1 [==============================] - 0s 143ms/step
[0.23193218]


# Combine the models

In [16]:
b = argument(Tensor(np.float32, ('N',45)))
print(b)
(r,) = inline(onnx_model)(b).values()
(r1,) = inline(onnx_model1)(b).values()
print(r)
print(r1)

r = op.div(op.add(r, r1), op.constant(value_float=2))

<Var from spox.internal@0::Argument->arg of float32[N][45]>
<Var from spox.internal@0::Inline->outputs_0 of float32[?]>
<Var from spox.internal@0::Inline->outputs_0 of float32[?]>


In [17]:
model_combine = build({'args_0': b}, {'average_w': r})
print(type(model_combine), type(onnx_model), type(onnx_model1))
# model_combine is a ModelProto object
#save the model
onnx.save(model_combine, "./model_combine.onnx")

<class 'onnx.onnx_ml_pb2.ModelProto'> <class 'onnx.onnx_ml_pb2.ModelProto'> <class 'onnx.onnx_ml_pb2.ModelProto'>


In [18]:
session_combine = rt.InferenceSession(
    "./model_combine.onnx",
    providers=rt.get_available_providers()
)
# print the input/putput name and shape
input_name=[input.name for input in session_combine.get_inputs()]
output_name=[output.name for output in session_combine.get_outputs()]
print("Inputs name:", input_name)
print("Outputs name:", output_name)

input_shape=[input.shape for input in session_combine.get_inputs()]
output_shape=[output.shape for output in session_combine.get_outputs()]
print("Inputs shape:", input_shape)
print("Outputs shape:", output_shape)

input_example = {input_name[0]: [input_data]}

output = session_combine.run(output_name, input_example)
print(output)


Inputs name: ['args_0']
Outputs name: ['average_w']
Inputs shape: [['N', 45]]
Outputs shape: [[None]]
[array([3.770484], dtype=float32)]


In [24]:
print((pred_ratio+pred_ratio1)/2)
print((pred_ratio+pred_ratio1+pred_ratio2)/3)

[3.7704892]
[2.5909703]


In [25]:
session_combine = rt.InferenceSession(
    main_dir+"/average_model012.onnx",
    providers=rt.get_available_providers()
)
# print the input/putput name and shape
input_name=[input.name for input in session_combine.get_inputs()]
output_name=[output.name for output in session_combine.get_outputs()]
print("Inputs name:", input_name)
print("Outputs name:", output_name)

input_shape=[input.shape for input in session_combine.get_inputs()]
output_shape=[output.shape for output in session_combine.get_outputs()]
print("Inputs shape:", input_shape)
print("Outputs shape:", output_shape)

input_example = {input_name[0]: [input_data]}

output = session_combine.run(output_name, input_example)
print(output)


Inputs name: ['args_0']
Outputs name: ['avg_w']
Inputs shape: [['N', 45]]
Outputs shape: [[None]]
[array([2.5909665], dtype=float32)]
